## Training a Wide ResNet with Dask Classifier

- Rerun LR decrease starting with 0.05
- Rerun BS increase starting with 0.05
- Run 2nd/3rd hybrid exps

Make simple dist.py that reads from CSV. 
- Subclass dask classifier to return based on training CSV
- sleep during partial_fit, override to sleep on diff workers
- Overwrite score and _get_grads ( to sleep )
--- Function called 

In [6]:
from dask.distributed import Client

In [4]:
import os
import importlib  
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
from copy import copy

In [31]:
# import simulator
os.chdir('/home/ubuntu/adadamp-experiments/')
classifier = importlib.import_module("exp-dask.classifier")
from classifier import DaskClassifierExpiriments

In [32]:
# training client
from dask.distributed import Client
from dask.distributed import performance_report

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://172.31.40.124/8040/10 Dashboard: http://172.31.40.124:45029/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [35]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [36]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [37]:
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [38]:
def write_stats(hist, exp, epoch):
    toCSV = hist
    with open('./exp-dask/stats/{}/exp-{}.csv'.format(exp, epoch), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [39]:
def test(epoch, model, test_set):
    """
    tests if model meets certain testing standards:
    - by 10th epoch, accuracy is over 70%
    """
    if epoch == 10:
        print("[TEST] Testing accuracy for 10th epoch is over 70%")
        score = model.score(test_set)
        acc = model._meta['score__acc']
        if acc < 0.70:
            print("[TEST] Test failed with {} accuracy".format(acc))
            return False
        else:
            print("[TEST] Test passed with {} accuracy".format(acc))
    
    return True

In [40]:
def train(model, train_set, test_set, n_epochs=200, dampings=None, log_interval=1, exp='increasing-bs'):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    
    history = []
    for epoch in range(n_epochs):
        # check for updates
        bs = dampings.get(epoch, None)
        if bs:
            model.damping_ = bs
            print("[UPDATE] Updated model params")
            if epoch != 0:
                print("[UPDATE] Running loss on train set...", end=" ")
                train_score = model.score(train_set)
                print(model._meta['score__loss'], '(acc:', model._meta['score__acc'], ')')
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {}".format(score))
        history.append(datum)
        
        # test
        if test(epoch, model, test_set) == False:
            print("[TEST] Test failed, exiting")
            break
        
        if epoch % log_interval == 0:
            write_stats(history, exp, 'ep{}'.format(epoch))

    return history

In [41]:
# lr and bs
orig_lr = 0.05
orig_bs = 128
# exp
exp_lr = orig_lr * 2
exp_bs = orig_bs * 2
exp_momentum = 0.9
# damping sched
dampings = {
    0:   exp_bs,
    60:  exp_bs * 5,
    120: exp_bs * 5 * 5,
    180: exp_bs * 5 * 5 * 5,
}

device = "cpu" if not torch.cuda.is_available() else "cuda:0"
args = dict(
    module=Wide_ResNet,
    module__depth=16,
    module__widen_factor=4,
    module__dropout_rate=0.3,
    module__num_classes=len(classes),
    loss=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    optimizer__lr=exp_lr,
    optimizer__momentum=exp_momentum,
    optimizer__nesterov=True,
    optimizer__weight_decay=0.5e-3,
    max_epochs=200,
    device=device,
    grads_per_worker=128,
    client=client,
    max_batch_size=5120,
    lr=exp_lr,
    batch_size=exp_bs
)
# set up
model = DaskClassifierExpiriments(**args)
# train
hist = None
with performance_report(filename="dask-report.html"):
    hist = train(model, 
             train_set, 
             test_set, 
             n_epochs=200, 
             dampings=dampings, 
             log_interval=20,
             exp='large-bs-0'
            )
write_stats(hist, 'large-bs-0', 'final')

[UPDATE] Updated model params
[Epoch 0] Score: 0.5884999632835388
[Epoch 1] Score: 0.6516000032424927
[Epoch 2] Score: 0.7285000085830688
[Epoch 3] Score: 0.7662000060081482
[Epoch 4] Score: 0.7896999716758728
[Epoch 5] Score: 0.7972999811172485
[Epoch 6] Score: 0.8024999499320984
[Epoch 7] Score: 0.8089999556541443
[Epoch 8] Score: 0.8148999810218811
[Epoch 9] Score: 0.8120999932289124
[Epoch 10] Score: 0.8294000029563904
[TEST] Testing accuracy for 10th epoch is over 70%
[TEST] Test passed with 0.8315999507904053 accuracy
[Epoch 11] Score: 0.8185999989509583
[Epoch 12] Score: 0.8331999778747559
[Epoch 13] Score: 0.8416999578475952
[Epoch 14] Score: 0.8327999711036682
[Epoch 15] Score: 0.832099974155426
[Epoch 16] Score: 0.8470999598503113
[Epoch 17] Score: 0.8425999879837036
[Epoch 18] Score: 0.8384999632835388
[Epoch 19] Score: 0.8499999642372131
[Epoch 20] Score: 0.833299994468689
[Epoch 21] Score: 0.849399983882904
[Epoch 22] Score: 0.839199960231781
[Epoch 23] Score: 0.8438000082

/home/ubuntu/adadamp-experiments/adadamp/adadamp/_dist.py:175: UserWarning: Model appears not to update with diff=1.1543497176882197e-09
  warn(f"Model appears not to update with diff={rel_error}")


 Score: 0.8567999601364136
[Epoch 56] Score: 0.8640999794006348
[Epoch 57] Score: 0.8637999892234802
[Epoch 58] Score: 0.8673999905586243
[Epoch 59] Score: 0.8592000007629395
[UPDATE] Updated model params
[UPDATE] Running loss on train set... 0.25090590240478516 (acc: 0.914359986782074 )
[Epoch 60] Score: 0.898099958896637
[Epoch 61] Score: 0.9013999700546265
[Epoch 62] Score: 0.9048999547958374
[Epoch 63] Score: 0.9041999578475952
[Epoch 64] Score: 0.9049999713897705
[Epoch 65] Score: 0.9027000069618225
[Epoch 66] Score: 0.9044999480247498
[Epoch 67] Score: 0.9060999751091003
[Epoch 68] Score: 0.9048999547958374
[Epoch 69] Score: 0.9027000069618225
[Epoch 70] Score: 0.903499960899353
[Epoch 71] Score: 0.905299961566925
[Epoch 72] Score: 0.9044999480247498
[Epoch 73] Score: 0.8973000049591064
[Epoch 74] Score: 0.9065999984741211
[Epoch 75] Score: 0.9038999676704407
[Epoch 76] Score: 0.9030999541282654
[Epoch 77] Score: 0.909500002861023
[Epoch 78] Score: 0.9001999497413635
[Epoch 79] S

In [42]:
print('done')

done


In [43]:
# lr and bs
orig_lr = 0.05
orig_bs = 128
# exp
exp_lr = orig_lr * 2
exp_bs = orig_bs * 2
exp_momentum = 0.98
# damping sched
dampings = {
    0:   exp_bs,
    60:  exp_bs * 5,
    120: exp_bs * 5 * 5,
    180: exp_bs * 5 * 5 * 5,
}
device = "cpu" if not torch.cuda.is_available() else "cuda:0"
args = dict(
    module=Wide_ResNet,
    module__depth=16,
    module__widen_factor=4,
    module__dropout_rate=0.3,
    module__num_classes=len(classes),
    loss=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    optimizer__lr=exp_lr,
    optimizer__momentum=exp_momentum,
    optimizer__nesterov=True,
    optimizer__weight_decay=0.5e-3,
    max_epochs=200,
    device=device,
    grads_per_worker=128,
    client=client,
    max_batch_size=5120,
    lr=exp_lr,
    batch_size=exp_bs
)
# set up
model = DaskClassifierExpiriments(**args)
# train
hist = None
with performance_report(filename="dask-report.html"):
    hist = train(model, 
             train_set, 
             test_set, 
             n_epochs=200, 
             dampings=dampings, 
             log_interval=20,
             exp='large-bs-1'
            )
write_stats(hist, 'large-bs-1', 'final')

[UPDATE] Updated model params
[Epoch 0] Score: 0.47829997539520264
[Epoch 1] Score: 0.5977999567985535
[Epoch 2] Score: 0.665399968624115
[Epoch 3] Score: 0.6798999905586243
[Epoch 4] Score: 0.6807999610900879
[Epoch 5] Score: 0.7182999849319458
[Epoch 6] Score: 0.7193999886512756
[Epoch 7] Score: 0.7482999563217163
[Epoch 8] Score: 0.6961999535560608
[Epoch 9] Score: 0.7603999972343445
[Epoch 10] Score: 0.7512999773025513
[TEST] Testing accuracy for 10th epoch is over 70%
[TEST] Test passed with 0.7529999613761902 accuracy
[Epoch 11] Score: 0.7337999939918518
[Epoch 12] Score: 0.7601000070571899
[Epoch 13] Score: 0.7540000081062317
[Epoch 14] Score: 0.7477999925613403
[Epoch 15] Score: 0.7642999887466431
[Epoch 16] Score: 0.7534999847412109
[Epoch 17] Score: 0.722000002861023
[Epoch 18] Score: 0.7458999752998352
[Epoch 19] Score: 0.7356999516487122
[Epoch 20] Score: 0.7387999892234802
[Epoch 21] Score: 0.7529000043869019
[Epoch 22] Score: 0.7604999542236328
[Epoch 23] Score: 0.7419999